# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse      Shipments             Customer  \
0  Mar  9 2023  2:25PM  259038        10      MSP219363     Methapharm, Inc.   
1  Mar  9 2023  2:25PM  259038        10      MSP219362     Methapharm, Inc.   
2  Mar  9 2023  2:25PM  259038        10      MSP219364         Methapharm-G   
3  Mar  9 2023  1:48PM  259035        10  Yusen-9248713    Yusen – 3D Matrix   
4  Mar  9 2023  1:48PM  259035        10  Yusen-9248717    Yusen – 3D Matrix   
5  Mar  9 2023  1:24PM  259032        19    ADV80013360  AdvaGen Pharma, Ltd   
6  Mar  9 2023  1:15PM  259030        10      MSP219356     Methapharm, Inc.   
7  Mar  9 2023  1:15PM  259030        10      MSP219357     Methapharm, Inc.   
8  Mar  9 2023  1:15PM  259030        10      MSP219359     Methapharm, Inc.   
9  Mar  9 2023  1:15PM  259030        10      MSP219360     Methapharm, Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
23  259022      Executing          1
24  259030       Released          5
25  259032       Released          1
26  259035       Released          2
27  259038       Released          3

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
259022                NaN        1.0       NaN
259030                NaN        NaN       5.0
259032                NaN        NaN       1.0
259035                NaN        NaN       2.0
259038                NaN        NaN       3.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
258960                0.0        0.0       1.0
258964                0.0        7.0       0.0
258975                0.0        0.0       1.0
258979                0.0        0.0       1.0
258980               29.0       23.0       5.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
258960                  0          0         1
258964                  0          7         0
258975                  0          0         1
258979                  0          0         1
258980                 29         23         5

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               258960          0          0         1
1               258964          0          7         0
2               258975          0          0         1
3               258979          0          0         1
4               258980         29         23         5

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               258960                            1
1               258964                   7         
2               258975                            1
3               258979                            1
4               258980        29        23        5

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                      Customer
0   Mar  9 2023  2:25PM  259038        10              Methapharm, Inc.
2   Mar  9 2023  2:25PM  259038        10                  Methapharm-G
3   Mar  9 2023  1:48PM  259035        10             Yusen – 3D Matrix
5   Mar  9 2023  1:24PM  259032        19           AdvaGen Pharma, Ltd
6   Mar  9 2023  1:15PM  259030        10              Methapharm, Inc.
11  Mar  9 2023 12:47PM  258997        15         Alliance Pharma, Inc.
54  Mar  9 2023 12:29PM  259022        10  Snap Medical Industries, LLC
55  Mar  9 2023 12:25PM  259020        21             NBTY Global, Inc.
56  Mar  9 2023 12:16PM  259019        12              SD Head USA, LLC
57  Mar  9 2023 12:15PM  259018        12                     Hush Hush

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                      Customer  \
0  Mar  9 2023  2:25PM  259038        10              Methapharm, Inc.   
1  Mar  9 2023  2:25PM  259038        10                  Methapharm-G   
2  Mar  9 2023  1:48PM  259035        10             Yusen – 3D Matrix   
3  Mar  9 2023  1:24PM  259032        19           AdvaGen Pharma, Ltd   
4  Mar  9 2023  1:15PM  259030        10              Methapharm, Inc.   
5  Mar  9 2023 12:47PM  258997        15         Alliance Pharma, Inc.   
6  Mar  9 2023 12:29PM  259022        10  Snap Medical Industries, LLC   
7  Mar  9 2023 12:25PM  259020        21             NBTY Global, Inc.   
8  Mar  9 2023 12:16PM  259019        12              SD Head USA, LLC   
9  Mar  9 2023 12:15PM  259018        12                     Hush Hush   

  Completed Executing Released  
0                            3  
1                            3  
2                            2  
3                            1  
4                            5  
5                           43  
6                   1           
7                            1  
8                            1  
9                   1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                      Customer  \
0  Mar  9 2023  2:25PM  259038        10              Methapharm, Inc.   
1  Mar  9 2023  2:25PM  259038        10                  Methapharm-G   
2  Mar  9 2023  1:48PM  259035        10             Yusen – 3D Matrix   
3  Mar  9 2023  1:24PM  259032        19           AdvaGen Pharma, Ltd   
4  Mar  9 2023  1:15PM  259030        10              Methapharm, Inc.   
5  Mar  9 2023 12:47PM  258997        15         Alliance Pharma, Inc.   
6  Mar  9 2023 12:29PM  259022        10  Snap Medical Industries, LLC   
7  Mar  9 2023 12:25PM  259020        21             NBTY Global, Inc.   
8  Mar  9 2023 12:16PM  259019        12              SD Head USA, LLC   
9  Mar  9 2023 12:15PM  259018        12                     Hush Hush   

  Released Executing Completed  
0        3                      
1        3                      
2        2                      
3        1                      
4        5                      
5       43                      
6                  1            
7        1                      
8        1                      
9                  1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse             Customer Released  \
0  Mar  9 2023  2:25PM  259038        10     Methapharm, Inc.        3   
1  Mar  9 2023  2:25PM  259038        10         Methapharm-G        3   
2  Mar  9 2023  1:48PM  259035        10    Yusen – 3D Matrix        2   
3  Mar  9 2023  1:24PM  259032        19  AdvaGen Pharma, Ltd        1   
4  Mar  9 2023  1:15PM  259030        10     Methapharm, Inc.        5   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Mar  9 2023  2:25PM  259038        10     Methapharm, Inc.       3.0   
1  Mar  9 2023  2:25PM  259038        10         Methapharm-G       3.0   
2  Mar  9 2023  1:48PM  259035        10    Yusen – 3D Matrix       2.0   
3  Mar  9 2023  1:24PM  259032        19  AdvaGen Pharma, Ltd       1.0   
4  Mar  9 2023  1:15PM  259030        10     Methapharm, Inc.       5.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Mar  9 2023  2:25PM  259038        10     Methapharm, Inc.       3.0   
1  Mar  9 2023  2:25PM  259038        10         Methapharm-G       3.0   
2  Mar  9 2023  1:48PM  259035        10    Yusen – 3D Matrix       2.0   
3  Mar  9 2023  1:24PM  259032        19  AdvaGen Pharma, Ltd       1.0   
4  Mar  9 2023  1:15PM  259030        10     Methapharm, Inc.       5.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3367066      22.0       28.0        1.0
12          518037       1.0        1.0        0.0
15          258997      43.0        0.0        0.0
19         1553944      10.0       47.0       37.0
21          259020       1.0        0.0        0.0
22          258993       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3367066      22.0       28.0        1.0
1        12   518037       1.0        1.0        0.0
2        15   258997      43.0        0.0        0.0
3        19  1553944      10.0       47.0       37.0
4        21   259020       1.0        0.0        0.0
5        22   258993       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      22.0       28.0        1.0
1        12       1.0        1.0        0.0
2        15      43.0        0.0        0.0
3        19      10.0       47.0       37.0
4        21       1.0        0.0        0.0
5        22       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   22.0
1        12  Released    1.0
2        15  Released   43.0
3        19  Released   10.0
4        21  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15    19   21   22
Status                                    
Completed   1.0  0.0   0.0  37.0  0.0  0.0
Executing  28.0  1.0   0.0  47.0  0.0  0.0
Released   22.0  1.0  43.0  10.0  1.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15    19   21   22
0          Completed   1.0  0.0   0.0  37.0  0.0  0.0
1          Executing  28.0  1.0   0.0  47.0  0.0  0.0
2           Released  22.0  1.0  43.0  10.0  1.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15    19   21   22
0  Completed   1.0  0.0   0.0  37.0  0.0  0.0
1  Executing  28.0  1.0   0.0  47.0  0.0  0.0
2   Released  22.0  1.0  43.0  10.0  1.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()